In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_wrapper, .output {height:auto;}</style>"))

C:\Users\Ilham\AppData\Local\Temp\ipykernel_13428\3540611931.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine

pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [3]:
def fetch_page(link):
    try:
        response = requests.get(link)
        if response.status_code == 200:
            print(f'HTTP request ke {link} berhasil')
            return response.content
        else:
            print(f'HTTP request ke {link} gagal dengan kode status {response.status_code}')
            return None
    except Exception as e:
        print(f'Terjadi kesalahan saat mengakses {link}: {e}')
        return None


In [33]:
# def scrape_items(page_content):
#     soup = BeautifulSoup(page_content, 'html.parser')
#     Items = []
#     Prices = []
#     # scraping
#     ItemDesc = soup.find_all('p', {'class': 'bl-text bl-text--body-14 bl-text--ellipsis__2'})
#     ItemPrice = soup.find_all('p', {'class': 'bl-text bl-text--subheading-20 bl-text--semi-bold bl-text--ellipsis__1'})
#     for Item in ItemDesc:
#         Items.append(Item.text.strip())
#     for Price in ItemPrice:
#         Prices.append(Price.text.strip())
#     return Items, Prices


def scrape_items(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    Items = []
    Prices = []
    Ratings = []
    
    product_list = soup.find_all('div', {'class': 'bl-product-card'})

    for product in product_list:
        Item_Desc = product.find('p', {'class': 'bl-text bl-text--body-14 bl-text--ellipsis__2'})
        Item_Price = product.find('p', {'class': 'bl-text bl-text--subheading-20 bl-text--semi-bold bl-text--ellipsis__1'})
        # Item_Rating = product.find(['p', 'div'], {'class': 'bl-text bl-text--body-14 bl-text--subdued', 'class': 'bl-product-card__description-rating'}) # like inner join
        Item_Rating = product.find('p', {'class': 'bl-text bl-text--body-14 bl-text--subdued'})
        
        if Item_Desc and Item_Price and Item_Rating:
            Items.append(Item_Desc.text.strip())
            Prices.append(Item_Price.text.strip())
            if Item_Rating:
                Item_Rating = Item_Rating.text.strip()
                # check "terjual"
                if "terjual" in Item_Rating.lower():
                    Ratings.append(None)
                else:
                    Ratings.append(Item_Rating)
            else:
                    Ratings.append(None)
            # Ratings.append(Item_Rating.text.strip())
    
    return Items, Prices, Ratings



In [34]:
baselink = 'https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat'
baselinkpg = 'https://www.bukalapak.com/products?page={}&search%5Bkeywords%5D=sepeda%20lipat'

# save all pages
all_Items = []
All_Prices = []
All_Ratings = []

for i in range(1, 11):
    if i == 1:
        link = baselink
    else:
        link = baselinkpg.format(i)
    
    page_content = fetch_page(link)
    
    if page_content:
        Items, Prices, Ratings = scrape_items(page_content)
        all_Items.extend(Items)
        All_Prices.extend(Prices)
        All_Ratings.extend(Ratings)


HTTP request ke https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=2&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=3&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=4&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=5&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=6&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=7&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=8&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=9&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=10&search%5Bkeywords%5D=

In [35]:
df = pd.DataFrame({
    'ItemName': all_Items,
    'Price': All_Prices,
    'Rating': All_Ratings
})

# Print DataFrame
print(df)

                                                                                              ItemName  \
0                                  Paket Kabel Shifter TRLREQ Set Sepeda Lipat MTB Roadbike Front Rear   
1                                                  Sepeda 16 Lipat Odessy Ritz 7Speeda Cakram New 2021   
2                                                               Sepeda  Lipat 20 Odessy Ritz Boncengan   
3                                                     Sepeda 16 Lipat Odessy Phyton 7Speeds Cakram New   
4                                     Sepeda Lipat 16 inch Pacific 2980 RX 9.0 Folding Bike 2980RX 9.0   
5                                           Lankeleisi Sepeda Elektrik Lipat Moped Elite Version XT750   
6                                     SEPEDA LIPAT FOLDING BIKE ELEMENT ECOSMO Z8 BLACK PANTHER MARVEL   
7                                        Sepeda Lipat Genio Runes 1.0 New chromoly 20 inch dan 16 inch   
8                                  Xiaomi HIMO

In [92]:
baselink = 'https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat'
baselinkpg = 'https://www.bukalapak.com/products?page={}&search%5Bkeywords%5D=sepeda%20lipat'

listresponse = []
for i in range(1,4):
    if i == 1:
        link = baselink.format(i)
        try:
            response = requests.get(link)
            if response.status_code == 200:
                print('requests http terhadap link ' + str(i) + ' berhasil')
                print(link)
            else:
                print('requests http terhadap link ' + str(i) + ' berhasil tetapi perlu hak akses')
                print(link)
        except:
            print('requests http terhadap link ' + str(i) + ' tidak berhasil')
            print(link)
    else:
        linkpg = baselinkpg.format(i)
        try:
            response = requests.get(linkpg)
            if response.status_code == 200:
                print('requests http terhadap linkpg ' + str(i) + ' berhasil')
                print(linkpg)
            else:
                print('requests http terhadap linkpg ' + str(i) + ' berhasil tetapi perlu hak akses')
                print(linkpg)
        except:
            print('requests http terhadap linkpg ' + str(i) + ' tidak berhasil')
            print(linkpg)

if response.status_code == 200:
    ItemDesc = soup.find_all('p', {'class': 'bl-text bl-text--body-14 bl-text--ellipsis__2'})
    # ItemName.text.strip()
    items = []
    for item in ItemDesc:
        items.append(item.text.strip())

df = pd.DataFrame(items, columns=['ItemName'])

print(df)


requests http terhadap link 1 berhasil
https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat
requests http terhadap linkpg 2 berhasil
https://www.bukalapak.com/products?page=2&search%5Bkeywords%5D=sepeda%20lipat
requests http terhadap linkpg 3 berhasil
https://www.bukalapak.com/products?page=3&search%5Bkeywords%5D=sepeda%20lipat
                                                                                                  ItemName
0                                              Seatpost ORI SEPEDA LIPAT ALLOY CNC Ultralight 33.9mmx600mm
1              Sadel Sepeda Super Empuk Kursi Jok MTB Gunung Lipat Saddle Handle Bike Road Ventilasi Udara
2                                      Xiaomi HIMO Z20 Sepeda Lipat Elektrik Smart Moped Bicycle 250W 80KM
3               Sepeda Lankeleisi G650 Elektrik Lipat Smart Moped Deluxe Version 48V 10.4Ah - Black-Yellow
4                                             Sepeda Lipat Exotic Second Sepeda Lipat Bekas United Pasific
5         

In [9]:
# Define connection
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-B14RR7Q, 1433;" # ConnectionName and port
    "DATABASE=Abstract;"
    "UID=sa;"
    "PWD=123;"
)

# Create a connection
conn = pyodbc.connect(conn_str)


In [27]:
table_name = 'Province'

for index, row in df.iterrows():
    sql = f"INSERT INTO {table_name} (No, Provinsi, Kode, ID, Kep, Ibu_Kota, Wilayah_Geografis, Gubernur, Hari_Jadi, UU, Kab, Kota) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    conn.execute(sql, row['No'], row['Provinsi'], row['Kode'], row['ID'], row['Kep.'], row['Ibu kota'], row['Wilayah geografis'], row['Gubernur'], row['Hari jadi'], row['UU'], row['Kab'], row['Kota'])

conn.commit()

conn.close()


In [35]:
## Insert langsung ke table nya tanpa harus create dulu
server = 'DESKTOP-B14RR7Q,1433'
database = 'Abstract'
username = 'sa'
password = '123'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'

engine = create_engine(connection_string)

table_name = 'Test'
df.to_sql(table_name, con=engine, if_exists='append', index=False)

print("Data inserted successfully!")


Data inserted successfully!
